In [ ]:
#!pip install opencv_transforms
import argparse
import os
import sys
import numpy as np
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt

import torch
import torch.autograd as autograd
import torch.optim as optim

import cv2
from datasets.test_aug_SR import SRDataset

from opencv_transforms import transforms as transform_cv2
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader

data = SRDataset(dataroot='/data/data/imageSR/vecky400/train/')
train_dataset = DataLoader(data, batch_size=4,shuffle=True)

from torchvision import utils
for i_batch, sample in enumerate(train_dataset):
    if 1:#i_batch == 80:
        img = sample['A'].numpy()        
        img2 = img.transpose(1,2,0).copy()
        
        cv2.rectangle(img2,(int(256*bbox[0]), int(256*bbox[1])),( int(256*bbox[0] + 256*bbox[2]), int(256*bbox[1] + 256*bbox[3])),(0,255,255), 2)

        fig = plt.figure()
        img = img2.transpose(2,0,1)
        sample['image'] = torch.from_numpy(img)
        grid = utils.make_grid(sample['image'])
        plt.imshow(grid.numpy().transpose((1, 2, 0)))
        plt.show()
        #break

In [ ]:
import data.ffhqSR_dataset
# import data.aligned_hair_face_dataset
print(1)

In [ ]:
import os 
os.system('sh /data/pytorch-CycleGAN-and-pix2pix-master/train.sh \
--crop_size 256 \
--model defaultpix2pix \
--name test_ffhqSuperResolution \
--ngf 16 \
--ndf 64 \
--gan_mode vanilla \
--netD basic \
--dataset_mode ffhqSR \
--norm batch \
--batch_size 200 \
--gpu_ids 0,1,2,3,4,5,6,7\
--netG unet_256 \
--load_memory \
--lambda_L1 100.0 \
--dataroot /data/data/ffhqSR \
1>/data/Log/ffhqSR/log1.log 2>/data/Log/ffhqSR/err1.log')

In [ ]:
from models.loss import ssim
import os
from PIL import Image
from torch.autograd import Variable
import torchvision.utils as vutils
import torchvision.transforms as transforms
import cv2
# 载入数据
# compute loss def ssim(img1, img2):
mean = [0.5, 0.5, 0.5] 
std = [0.5, 0.5, 0.5] 
pil_transforms = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=mean, std=std),
            ])

def trans_data(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = img[120-60:120+270,102+10:102+265-10,:]
    img = cv2.resize(img, (256, 256)) 
    img = Image.fromarray(np.uint8(img))
    w, h = img.size
    img = pil_transforms(img)        
    return img

dataroot = '/data/facecyclegan/pairs_for_distill/step2_crop7_male2beauty_stylegan_cyc50_vgg10'
img_files = os.listdir(dataroot)[:200]
img_files_list = [dataroot + f for f in img_files]

out_dir = '/data/data/forTest'
# for test
img_files_list = img_files_list[0]

for path in img_files_list:
    print(path)
    img = trans_data(path)  # crop掉一半
    tensor = torch.unsqueeze(img, dim=0).float()
    vutils.save(tensor, out_dir + '/test.png')
